In [1]:
;;
;; Cria uma base de conhecimento (KB)
(define (create-kb rules facts)
  (let ((table-rules rules)
        (table-facts facts))
    (define (dispatch query)
      (cond ((eq? query 'rules) table-rules)
            ((eq? query 'facts) table-facts)
            (else (error "Unknown KB query:" query))))
    dispatch))

;; Adiciona uma regra (premissa . conclusão)
(define (add-rule kb premise conclusion)
  (let* ((rules (kb 'rules))
         (new-rules (cons (cons premise conclusion) rules)))
    (create-kb new-rules (kb 'facts))))

;; Adiciona um fato simples (booleano)
(define (add-fact kb sym)
  (let* ((facts (kb 'facts))
         (new-facts (cons (cons sym #t) facts)))
    (create-kb (kb 'rules) new-facts)))

;; Adiciona um fato com valor associado
(define (add-fact-with-value kb sym val)
  (let* ((facts (kb 'facts))
         (new-facts (cons (cons sym val) facts)))
    (create-kb (kb 'rules) new-facts)))

;; Retorna o valor associado a um símbolo (ou #f)
(define (fact-value kb sym)
  (let ((entry (assq sym (kb 'facts))))
    (if entry (cdr entry) #f)))


In [2]:
;; Inferência iterativa — versão estável e segura

;; auxiliares
(define (any pred lst)
  (cond ((null? lst) #f)
        ((pred (car lst)) #t)
        (else (any pred (cdr lst)))))

(define (every pred lst)
  (cond ((null? lst) #t)
        ((pred (car lst)) (every pred (cdr lst)))
        (else #f)))

(define (filter pred lst)
  (cond ((null? lst) '())
        ((pred (car lst)) (cons (car lst) (filter pred (cdr lst))))
        (else (filter pred (cdr lst)))))

In [3]:
;;
(define (infer-loop4 kb goal)
  (let loop ((known (kb 'facts)))
    ;; verifica se goal já é fato
    (if (assq goal known)
        #t
        (let* ((rules (kb 'rules))
               (applicable
                (filter
                 (lambda (r)
                   (let ((prem (car r)))
                     (cond
                      
                       ;; OR : Alguma premissa precisa ser verdadeira 
                       ((and (pair? prem) (eq? (car prem) 'or))
                        (any (lambda (p) (assq p known)) (cdr prem)))

                       ;; AND : Todas as premissas precisam ser verdadeiras
                       ((and (pair? prem) (eq? (car prem) 'and))
                        (every (lambda (p) (assq p known)) (cdr prem)))

                       ;; OPERADOR / FUNÇÃO: somente se tiver pelo menos 2 elementos
                       ;; (evita avaliar '(C) e causar unbound-variable)
                       ((and (pair? prem)                     ; é lista
                             (pair? (cdr prem))              ; tem pelo menos 2 elementos
                             (symbol? (car prem)))           ; car é símbolo (nome do operador)
 
                        ;; tenta avaliar o car; se não for procedimento, trata como false
                        (let ((maybe-proc
                               (catch #t
                                 (lambda ()
                                   (eval (car prem) (interaction-environment)))
                                 (lambda args
                                   ;; falha ao avaliar (nome nao definido) -> retorne #f para non-proc
                                   #f))))
                          (if (and (procedure? maybe-proc))
                              (let* ((f maybe-proc)
                                     (args (cdr prem))
                                     ;; extrai valores de facts se existirem, caso contrario passa símbolo
                                     (vals (map (lambda (x)
                                                  (let ((entry (assq x known)))
                                                    (if entry (cdr entry) x)))
                                                args)))

                                (let ((res (apply f vals)))

                                  res))
                              ;; Se nao e uma func, trata como nao aplicavel
                              #f)))
                      
                       ;; lista simples — trata como AND (listas de 1 elemento como '(C))
                       ((list? prem)

                        (every (lambda (p) (assq p known)) prem))

                       ;; símbolo isolado
                       (else

                        (assq prem known)))))
                 rules))
               (conclusions (map cdr applicable))
               (to-add (filter (lambda (c) (not (assq c known))) conclusions))
               (new-known (append known (map (lambda (x) (cons x #t)) to-add))))
          ;; debug opcional por iteração

          (if (equal? new-known known)
              #f
              (loop new-known))))))


In [4]:
(define (infer-loop_dbg kb goal)
  (let loop ((known (kb 'facts)))
    ;; verifica se goal já é fato
    (if (assq goal known)
        #t
        (let* ((rules (kb 'rules))
               (applicable
                (filter
                 (lambda (r)
                   (let ((prem (car r)))
                     (cond
                       ;; caso or -> usa any pra ver se existe uma premissa que e fato
                       ((and (pair? prem) (eq? (car prem) 'or))
                        (display "[DEBUG] Testando OR: ") (display prem) (newline)
                        (any (lambda (p) (assq p known)) (cdr prem)))

                       ;; caso and -> usa every pra ver se todas as premissas sao fatos
                       ((and (pair? prem) (eq? (car prem) 'and))
                        (display "[DEBUG] Testando AND: ") (display prem) (newline)
                        (every (lambda (p) (assq p known)) (cdr prem)))

                       ;; caso operador/funcao -> somente se tiver pelo menos 2 elementos
                       ;; (evita avaliar '(C) e causar unbound-variable)
                       ((and (pair? prem)                     ; é lista
                             (pair? (cdr prem))              ; tem pelo menos 2 elementos
                             (symbol? (car prem)))           ; car é símbolo (nome do operador)
                        (display "[DEBUG] Possível operador: ") (display prem) (newline)
                        ;; tenta avaliar o car; se não for procedimento, trata como false
                        (let ((maybe-proc
                               (catch #t
                                 (lambda ()
                                   (eval (car prem) (interaction-environment)))
                                 (lambda args
                                   ;; falha ao avaliar (nome nao definido) -> retorne #f para non-proc
                                   #f))))
                          (if (and (procedure? maybe-proc))
                              (let* ((f maybe-proc)
                                     (args (cdr prem))
                                     ;; extrai valores de facts se existirem, caso contrario passa símbolo
                                     (vals (map (lambda (x)
                                                  (let ((entry (assq x known)))
                                                    (if entry (cdr entry) x)))
                                                args)))
                                (display "    vals => ") (display vals) (newline)
                                (let ((res (apply f vals)))
                                  (display "    resultado => ") (display res) (newline)
                                  res))
                              ;; not a procedure (treat as not applicable)
                              #f)))

                       ;; lista simples — trata como AND (listas de 1 elemento como '(C))
                       ((list? prem)
                        (display "[DEBUG] Testando LISTA (AND implícito): ") (display prem) (newline)
                        (every (lambda (p) (assq p known)) prem))

                       ;; símbolo isolado
                       (else
                        (display "[DEBUG] Testando símbolo: ") (display prem) (newline)
                        (assq prem known)))))
                 rules))
               (conclusions (map cdr applicable))
               (to-add (filter (lambda (c) (not (assq c known))) conclusions))
               (new-known (append known (map (lambda (x) (cons x #t)) to-add))))
          ;; debug opcional por iteração
          (display "rules: ") (display rules) (newline)
          (display "known: ") (display known) (newline)
          (display "to-add: ") (display to-add) (newline)
          (if (equal? new-known known)
              #f
              (loop new-known))))))


In [5]:
;; MACROS
;; Fatos booleanos e com valores
(define-syntax fact
  (syntax-rules ()
    ((_ sym)
     (set! kb (add-fact kb 'sym)))
    ((_ sym val)
     (set! kb (add-fact-with-value kb 'sym val)))))

;; Regras logicas e funcionais 
(define-syntax rule
  (syntax-rules (and or =>)
    ((_ A and B => C)
     (set! kb (add-rule kb (quote (and A B)) (quote C))))
    ((_ A or B => C)
     (set! kb (add-rule kb (quote (or A B)) (quote C))))
    ((_ A op B => C)
     (set! kb (add-rule kb (list (quote op) (quote A) (quote B)) (quote C))))
    ((_ A => B)
     (set! kb (add-rule kb (list (quote A)) (quote B))))))





(define-syntax infer
  (syntax-rules ()
    ((_ sym)
     (infer-loop4 kb 'sym))))





In [7]:
;; ==========================
;;  Demonstração Completa
;; ==========================

(define kb (create-kb '() '()))

;; Fatos booleanos e com valores
(fact A)
(fact B)
(fact Temp 35)
(fact Limite 30)

;; Funções personalizadas
(define (xor a b)
  (or (and a (not b)) (and b (not a))))

(define (maior x y)
  (> x y))

;; Regras
(rule A and B => C)
(rule A xor B => D)
(rule Temp maior Limite => quente)
(rule quente => alerta)
(rule C => E)
(rule E and D => F)

;; Inferências
(display "Inferindo C (A and B): ") (display (infer C)) (newline)
(display "Inferindo D (A xor B): ") (display (infer D)) (newline)
(display "Inferindo quente (Temp > Limite): ") (display (infer quente)) (newline)
(display "Inferindo alerta: ") (display (infer alerta)) (newline)
(display "Inferindo F (E and D): ") (display (infer F)) (newline)


Inferindo C (A and B): #t
Inferindo D (A xor B): #f
Inferindo quente (Temp > Limite): #t
Inferindo alerta: #t
Inferindo F (E and D): #f
